In [1]:
# Setup
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import time
import warnings
import torch.nn as nn
import pickle
from dataset_loader import *
from utils import *
import pandas as pd
from datasets import Dataset, DatasetDict

SEED = 84
torch.manual_seed(SEED)
warnings.simplefilter("ignore")

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
dataset_name = 'corona'
ds = get_ds(dataset_name)

In [8]:
# 50 padding for sentiment, 64 for corona
set_seed()    
model_name = 'huawei-noah/TinyBERT_General_4L_312D'
folder_name = 'gru'
model = MyGRU(model_name, hidden_dim=256, num_layers=2, output_dim=2, dropout=0.0)
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}', num_train_epochs=20)

loading configuration file config.json from cache at /home/almr16/.cache/huggingface/hub/models--huawei-noah--TinyBERT_General_4L_312D/snapshots/34707a33cd59a94ecde241ac209bf35103691b43/config.json
Model config BertConfig {
  "_name_or_path": "huawei-noah/TinyBERT_General_4L_312D",
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "structure": [],
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/almr16/.cache/huggingface/hub/models--huawei-noah--TinyBERT_General_4L_312

  0%|          | 0/8000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

PyTorch: setting up devices
The following columns in the training set don't have a corresponding argument in `MyGRU.forward` and have been ignored: attention_mask, token_type_ids. If attention_mask, token_type_ids are not expected by `MyGRU.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 8000
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 5000


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693933,0.484500
2,0.693100,0.693382,0.517500
3,0.693100,0.693159,0.518500
4,0.692000,0.692254,0.519000
5,0.692000,0.688618,0.506000
6,0.619500,0.496389,0.760500
7,0.619500,0.458393,0.791500
8,0.337500,0.441090,0.806500
9,0.337500,0.477021,0.797000
10,0.209200,0.477959,0.809500


The following columns in the evaluation set don't have a corresponding argument in `MyGRU.forward` and have been ignored: attention_mask, token_type_ids. If attention_mask, token_type_ids are not expected by `MyGRU.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to gru/corona/checkpoint-250
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
The following columns in the evaluation set don't have a corresponding argument in `MyGRU.forward` and have been ignored: attention_mask, token_type_ids. If attention_mask, token_type_ids are not expected by `MyGRU.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32
Saving model checkpoint to gru/corona/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
The following columns in the evaluation set don't have a corresponding argument in `MyGR

TrainOutput(global_step=5000, training_loss=0.30256288032531736, metrics={'train_runtime': 87.4733, 'train_samples_per_second': 1829.131, 'train_steps_per_second': 57.16, 'total_flos': 0.0, 'train_loss': 0.30256288032531736, 'epoch': 20.0})

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /home/almr16/.cache/huggingface/hub/models--huawei-noah--TinyBERT_General_4L_312D/snapshots/34707a33cd59a94ecde241ac209bf35103691b43/config.json
Model config BertConfig {
  "_name_or_path": "huawei-noah/TinyBERT_General_4L_312D",
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "structure": [],
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /ho

In [10]:
def predict_sentences(sentences):
    encoded = [[101] +[tokenizer._convert_token_to_id_with_added_voc(token) for token in tokens] + [102]         
               for tokens in sentences]
    to_pred = torch.tensor(encoded, device=device)
    outputs = model(to_pred)[0]
    print(outputs)
    return torch.argmax(outputs, dim=1).cpu().numpy()

### save best model's folder as 'model'

In [32]:
model = load_gru(f'{folder_name}/{dataset_name}/model')
model = model.eval()

loading configuration file config.json from cache at /home/almr16/.cache/huggingface/hub/models--huawei-noah--TinyBERT_General_4L_312D/snapshots/34707a33cd59a94ecde241ac209bf35103691b43/config.json
Model config BertConfig {
  "_name_or_path": "huawei-noah/TinyBERT_General_4L_312D",
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "structure": [],
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/almr16/.cache/huggingface/hub/models--huawei-noah--TinyBERT_General_4L_312

In [ ]:
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)

In [12]:
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}', evaluate = True)

PyTorch: setting up devices
The following columns in the evaluation set don't have a corresponding argument in `MyGRU.forward` and have been ignored: attention_mask, token_type_ids. If attention_mask, token_type_ids are not expected by `MyGRU.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 32


{'eval_loss': 0.5787323117256165,
 'eval_accuracy': 0.818,
 'eval_runtime': 0.4803,
 'eval_samples_per_second': 4163.967,
 'eval_steps_per_second': 131.165}

In [13]:
encoded = tokenizer.encode("i love movies", add_special_tokens=True, return_tensors="pt").to(device)
traced_model = torch.jit.trace(model, encoded)

In [14]:
torch.jit.save(traced_model, f"{folder_name}/{dataset_name}/traced.pt")

In [15]:
traced_model = torch.jit.load(f"{folder_name}/{dataset_name}/traced.pt")

In [16]:
predict_sentences([tokenizer.tokenize("good")])

tensor([[-0.0229, -0.0089]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([1])

In [17]:
predict_sentences([tokenizer.tokenize("bad")])

tensor([[ 0.4185, -0.5113]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([0])